In [1]:
import numpy as np
import plotly.express as px
import dash

# for data import
import os
from glob import glob
import lz4.frame

import gzip
import nibabel as nib
import shutil

In [2]:
# import test brain volume
def load_image(path: str):
    if path.endswith('.npy.lz4'):
        with lz4.frame.open(path, 'rb') as f:
            return np.load(f)
    elif path.endswith('.npy'):
        return np.load(path)
    elif path.endswith('.nii.gz'):
        return nib.load(path).get_fdata() # open as numpy array
    else:
        raise Exception("File extension not supported!")

In [3]:
TEST_VOL = '/usr/local/data/kvasilev/mais/data/BraTS19_CBICA_ASK_1_t2.nii.gz'

In [4]:
# load data
data = load_image(TEST_VOL)

In [5]:
data.shape

(240, 240, 155)

In [12]:
data.dtype

dtype('float64')

In [22]:
np.max(data[75,:,:])

1228.0

In [17]:
data[40,:,:].shape

(240, 155)

In [28]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

# Plotting
# For reference: mri dims: dim0=saggital, dim1=coronal, dim2=axial
import chart_studio.plotly as py
import plotly.graph_objects as go

fig = px.imshow(data[:,:,80])
fig.show()

In [34]:
from datetime import date

# using dash
app = dash.Dash(__name__)

layout = [dcc.Graph(id='mri-plot'),
          html.P('MRI Visualizer'),
          dcc.Graph(id='mri-plot'),
          dcc.Slider(id='slice-slider', min=0,max=88,step=1,value=40, marks={0:'0', 88:'88'}),
          dcc.Slider(id='age-slider', min=60, max=80, marks={60: 'Age 60', 80:'Age 80'}, value=60),
          dcc.RadioItems(id='slice-selector',
                         options=[{'label':'Axial', 'value': 'axial'},
                                  {'label':'Coronal', 'value':'coronal'},
                                  {'label':'Saggital', 'value':'saggital'}],
                                 value='axial',
                                 labelStyle={'display':'inline-block'}),
          
          dcc.DatePickerSingle(id='bdate-picker',
                          date=date(1969,4,20)),
          html.Button('Submit', id='button')]

app.layout = html.Div(layout)

@app.callback(
    Output('mri-plot', 'figure'),
    [Input('slice-slider','value'),
     Input('age-slider','value'),
     Input('slice-selector', 'value'),
     Input('bdate-picker','date'),
     ])    # ADD BUTTON BACK
def visualize_mri(slice_num, age_num, slice_type, bdate_num):

    # plot appropriate slice
    if slice_type=='saggital': #dim0
        fig = px.imshow(data[slice_num,:,:])
    elif slice_type=='coronal': #dim1
        fig = px.imshow(data[:,slice_num,:])
    elif slice_type=='axial': #dim2
        fig = px.imshow(data[:,:,slice_num])
    else:
        print('Invalid axis selection!')
    
    return fig

# show         
app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/usr/local/data/kvasilev/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

